In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import pymr.heart as pyheart
from os.path import basename, join

In [3]:
from pymr import heart as pyheart
#pyheart.dataset.convert_mms(r'/home/OpenDataset', dest_dir=r'/home/OpenDataset/MMS_lab19')

In [ ]:
pyheart.dataset.convert_acdc(r'/home/ACDC/ACDC', dest_dir=r'/home/OpenDataset/ACDC_lab19')

pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
../pymr/heart/viz.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(10, 10))
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (defau

In [5]:
data = r'/home/OpenDataset/ACDC_lab19/patient009_gt.nii.gz'


In [6]:
import nibabel
pyheart.dataset.get_slice_label(nibabel.load(data).get_fdata())

array([1, 1, 1, 2, 2, 2, 3, 3, 4, 4])

In [ ]:
import numpy as np
heart = nibabel.load(data).get_fdata().copy()
curve = np.zeros((heart.shape[2],))
sys_frame, dia_frame = pyheart.get_frame(heart)
for slicen in range(heart.shape[2]):    
    heart_mask_xyt = heart[:, :, slicen, :]
    if np.unique(heart_mask_xyt[..., sys_frame]).sum() + np.unique(heart_mask_xyt[..., dia_frame]).sum() == 12:
        curve[slicen] = 1

curve_and = curve.astype(np.int)
curve_or = (np.sum(heart, axis=(0, 1, 3)) > 0).astype(np.int)
curve_diff = curve_or - curve_and
diff = np.diff(np.sum(heart==1, axis=(0, 1, 3)))
diff = np.median(diff[curve_and[1:] > 0])

slice_label = curve_and * 0
mid_point = curve_and.size//2

curve_apex = curve_diff.copy()
curve_basal = curve_diff.copy()

In [ ]:
curve_and = curve.astype(np.int)

In [ ]:
diff = np.diff(np.sum(heart==1, axis=(0, 1, 3)))
print(diff)

In [ ]:
diff = np.diff(np.sum(heart==1, axis=(0, 1, 3)))
diff = np.median(diff[curve_and[1:] > 0])
print(diff)

In [ ]:
temp = np.sum(heart==1, axis=(0, 1, 3))[curve_and>0]
diff = temp[-1] - temp[0]
print(diff)

In [ ]:
temp = np.sum(heart==1, axis=(0, 1, 3))
temp = temp[temp>0]
diff = temp[-1] - temp[0]
print(diff)

In [ ]:
curve_or = (np.sum(heart, axis=(0, 1, 3)) > 0).astype(np.int)
curve_diff = curve_or - curve_and
diff = np.diff(np.sum(heart==1, axis=(0, 1, 3))) * curve_and[1:]
diff = np.median(diff[curve_and[1:] > 0])

In [ ]:
if diff < 0:
    basal_first = True

    curve_apex[:mid_point] = 0
    curve_basal[mid_point:] = 0
else:
    basal_first = False
    curve_apex[mid_point:] = 0
    curve_basal[:mid_point] = 0

got_apex = np.sum(curve_apex) > 0

if basal_first:
    slice_label[curve_and > 0] = pyheart.dataset.get_loc(np.sum(curve_and), got_apex)
    slice_label[curve_apex > 0] = 4
    slice_label[curve_basal > 0] = 11
else:
    slice_label[curve_and > 0] = pyheart.dataset.get_loc(np.sum(curve_and), got_apex)[::-1]
    slice_label[curve_apex > 0] = 4
    slice_label[curve_basal > 0] = 11

In [ ]:
f = glob.glob(r'/home/ACDC/ACDC/training/**/*_4d.nii.gz')[0]
temp = nibabel.load(f)
vol4d = temp.get_fdata()
affine = temp.affine
heart = vol4d * 0
for gt in glob.glob(f.replace('_4d.nii.gz','*gt.nii.gz' )):
    gt_mask = nibabel.load(gt).get_fdata()
    frame = int(basename(gt).split('_')[1].replace('frame', ''))
    heart[..., frame] = gt_mask
    

In [ ]:
def safe_mkdir(dirname):
    try:
        os.mkdir(dirname)
    except Exception as e:
        pass

In [ ]:

import glob
import nibabel
import numpy as np
import shutil
import os
source_dir = r'/home/ACDC/ACDC'

dest_dir = join(source_dir, 'Convert_lab19') 
jpg_dir = join(dest_dir, 'jpg')

safe_mkdir(dest_dir)
safe_mkdir(jpg_dir)

for f in glob.glob(join(source_dir, 'training', '**', '*_4d.nii.gz')):
    
    try:
    #print(f)
        base_f = basename(f)
        temp = nibabel.load(f)
        vol4d = temp.get_fdata()
        affine = temp.affine
        heart = vol4d * 0
        for gt in glob.glob(f.replace('_4d.nii.gz','*gt.nii.gz' )):
            gt_mask = nibabel.load(gt).get_fdata()
            frame = int(basename(gt).split('_')[1].replace('frame', ''))
            heart[..., frame-1] = gt_mask
        heart_aha17_4d = pyheart.dataset.convert(heart, dataset='ACDC')

        result_f = join(dest_dir, base_f.replace('_4d.nii.gz', '_gt.nii.gz'))      
        nii_label = nibabel.Nifti1Image(heart.astype(np.uint8), affine)
        nibabel.save(nii_label, result_f)


        result_f = join(dest_dir, base_f.replace('_4d.nii.gz', '_lab19.nii.gz'))      
        nii_label = nibabel.Nifti1Image(heart_aha17_4d.astype(np.uint8), affine)
        nibabel.save(nii_label, result_f)


        result_f = base_f.replace('_4d.nii.gz', '.nii.gz')
        shutil.copyfile(f, join(dest_dir, result_f))

        sys_frame, dia_frame = pyheart.get_frame(heart)
        heart_crop, crop = pyheart.auto_crop(heart_aha17_4d)            
        jpg_file = join(jpg_dir, base_f + '.jpg')
        pyheart.montage(np.concatenate([heart_crop[..., sys_frame], heart_crop[..., dia_frame]], axis=-1), fname=jpg_file, display=False)
    
    except Exception as e:

        print(e)
        print('error:', f)
                

In [ ]:
pyheart.dataset.convert(heart, dataset='ACDC')

In [ ]:
    train_ffs = glob.glob(join(source_dir, 'Training', 'Labeled', '**', '*gt.nii.gz'))
    val_ffs = glob.glob(join(source_dir, 'Validation', '**', '*gt.nii.gz'))
    test_ffs = glob.glob(join(source_dir, 'Testing', '**', '*gt.nii.gz'))
    

    
    
    if dest_dir is None:
        dest_dir = join(source_dir, 'Convert_lab19')
        
        
    train_dir = join(dest_dir, 'Training')
    val_dir = join(dest_dir, 'Validation')
    test_dir = join(dest_dir, 'Testing')
    jpg_dir = join(dest_dir, 'jpg')
    safe_mkdir(dest_dir)
    safe_mkdir(train_dir)
    safe_mkdir(val_dir)
    safe_mkdir(test_dir)
    safe_mkdir(jpg_dir)
    
    loop = [(train_ffs, train_dir), (val_ffs, val_dir), (test_ffs, test_dir)]
    
    for ffs, result_dir in loop:
        print(result_dir)
        for f in ffs:
            try:
                heart_aha17_4d = convert(f, dataset='MMS', result_dir=result_dir)
                sys_frame, dia_frame = get_frame(nibabel.load(f).get_fdata())
                heart_crop, crop = auto_crop(heart_aha17_4d)            
                jpg_file = join(jpg_dir, basename(f) + '.jpg')
                montage(np.concatenate([heart_crop[..., sys_frame], heart_crop[..., dia_frame]], axis=-1), fname=jpg_file, display=False)
                img_file = f.replace('_gt.nii.gz', '.nii.gz')
                shutil.copyfile(img_file, join(result_dir, basename(img_file)))
                shutil.copyfile(f, join(result_dir, basename(f)))

            except Exception as e:

                print(e)
                print('error:', f)

In [ ]:
vol4d = temp.get_fdata()

In [ ]:
affine = temp.affine

In [ ]:
temp = nibabel.load(f)